# Como fazer chamadas simples com API? 

Antes de mais nada, vamos escolher o LLM utilizado, seja ele o ChatGPT (gepeto), o Gemini etc. Caso você escolha o gepeto, terá de fazer um **pip install openai**, caso escolha o gemini, terá de fazer **pip install google**. Uma vez escolhido o LLM, cria um ambiente virtual para isolar o ambiente de desenvolvimento do seu código com **python -m venv nome-do-seu-venv** e depois ative-o **.\nome-do-seu-venv\Scripts\Activate.ps1**. 

In [ ]:
# LLMs
from openai import OpenAI
from google import genai
from google.genai import types

# Para validação de campos e geração de uma resposta estruturada
from pydantic import BaseModel, Field
from enum import Enum
import json

# Para a leitura das chaves no arquivo ".env"
from dotenv import load_dotenv, dotenv_values




In [54]:
# Protegendo as chaves de API e carregando-as em variáveis
config = dotenv_values(".env")
openai_api_key = config["IRAN_OPENAI_KEY"]
gemini_api_key = config["IRAN_GEMINI_KEY"]

In [55]:
# Alimentando o client com as devidas chaves de api
client_openai = OpenAI(api_key=openai_api_key)
client_gemini = genai.Client(api_key=gemini_api_key)

In [ ]:
# Ambos as funções abaixo possuem o mesmo funcionamento: 
# 1 - Você passa um contexto/prompt (context)
# 2 - Você passa uma pergunta ou qulquer coisa que queira saber (input)
# As funções fazem uma chamada simples para os modelos de  LLM e a resposta obtida é retornada como resposta



# Função que utiliza a API da OpenAI
def response_openai(context:str, input:str) -> str:
    response = client_openai.responses.create(
        model="gpt-4.1-mini",
        instructions=context,
        input= input,
        temperature=0.5
    )
    return response.output_text




# Função que utiliza a API do Gemini
def response_gemini(context:str, input:str) -> str:
        
    response = client_gemini.models.generate_content(
        model="gemini-2.5-flash", 
        config=types.GenerateContentConfig(
            system_instruction=context,
            temperature=0.5),
        contents=input
    )
    return response.text

### Como obter uma resposta estruturada no formato de JSON? 

Vamos supor que você quer analisar um processo judicial envolvendo tráfico de drogas de forma rápida e eficiente. Para isso você precisa extrair informações como: 
- O processo se trata de tráfico de drogas?
- Qual o tipo de droga envolvida?
- Qual a quantidade do tipo de droga envolvida?
- Qual o sexo da pessoa envolvida? 
- Qual o tipo de pena? 
- Qual o tempo da pena em meses? 

Para isso precisaremos utilizar pydantic para garantir uma validação dos campos que devem ser retornados pelo LLM e precisamo mudar um pouquinho a estrutura das nossas funções de chamadas. 

In [57]:
# Set do tipo de resposta possível do LLM
class Escopo(Enum):
    SIM = "sim"
    NAO = "nao"


class Drogas(BaseModel):
    escopo: Escopo
    maconha: str
    cocaina: str
    crack: str
    qtd_maconha: float
    qtd_cocaina: float
    qtd_crack: float
    sexo: str
    reincidente: str
    decisao: str
    tipo_pena: str
    tempo: float


In [97]:
def classificador_openai(context:str, input:str, estrutura:Drogas) -> dict:
    response = client_openai.responses.parse(  # mudar aqui de .create() para .parse()
        model="gpt-4.1-mini",
        instructions=context,
        input=input,
        temperature=0.0,
        text_format = estrutura # mudar aqui para o schema de classe
    )
    # mudar aqui de "response.output_text" para "response.output_parsed"
    return response.output_parsed.model_dump()




def clasificador_gemini(prompt:str, input:str, estrutura=Drogas) -> dict:

    content = f"{prompt}\n\nProcesso:\n{input}"

    response = client_gemini.models.generate_content( # mudar aqui de ".generate_content()" para ".generate_content_stream()"
        model="gemini-2.5-flash",
        contents=content,
        config={
        "response_mime_type": "application/json",
        "response_json_schema": estrutura.model_json_schema(),
    },  
    )

    # mudar aqui de "response.text" por:
    result = response.candidates[0].content.parts[0].text
    return result

    

In [100]:
# É necessário também um bom prompt para garantir a saída de uma resposta que segue a estrutura que nós buscamos

prompt_otimizado = """
Você é um assistente de inteligência artificial que auxilia na anotação de sentenças judiciais do Tribunal de Justiça de São Paulo em processos envolvendo tráfico de drogas.

Você receberá o texto da sentença e deverá retornar um arquivo JSON, seguindo as regras abaixo:

- O processo faz parte do escopo da pesquisa? O caso deve: Ser um caso relacionado a tráfico de drogas; Envolver porte de maconha, crack ou cocaína; Envolver apenas uma pessoa acusada.
- Quantidade de maconha em gramas: Preencha apenas os números. Coloque 0 se o caso não envolve essa droga. Use "," como separador decimal. Se a decisão não menciona a quantidade em gramas, faça a conversão, seguindo a regra: 1 porção = 2 gramas.
- Quantidade de cocaína em gramas: Preencha apenas os números. Coloque 0 se o caso não envolve essa droga. Use "," como separador decimal. Se a decisão não menciona a quantidade em gramas, faça a conversão, seguindo a regra: 1 porção ou pino = 0,5 grama.
- Quantidade de crack em gramas: Preencha apenas os números. Coloque 0 se o caso não envolve essa droga. Use "," como separador decimal. Se a decisão não menciona a quantidade em gramas, faça a conversão, seguindo a regra: 1 porção = 0,1 grama.
- Decisão: pode ser procedente (condenação), improcedente / punibilidade extinta, ou parcialmente procedente / advertência.
- Tipo de pena: pode ser fechado, semiaberto ou aberto.
- Tempo da pena (em meses): Preencha apenas os números. Converta o tempo em meses. Por exemplo, 2 anos e 7 meses = 31 meses.

Retorne um arquivo JSON com as seguintes informações:

{
 "escopo": "sim/não",
 "maconha": "sim/não",
 "cocaina": "sim/não",
 "crack": "sim/não",
 "qtd_maconha": 0,
 "qtd_cocaina": 0,
 "qtd_crack": 0,
 "sexo": "masculino/feminino/não informado",
 "reincidente": "sim/não/não informado",
 "decisao": "procedente/improcedente/parcialmente procedente",
 "tipo_pena": "fechado/semiaberto/aberto",
 "tempo": 0
}
"""


{
 "escopo": "sim",
 "maconha": "sim",
 "cocaina": "nao",
 "crack": "nao",
 "qtd_maconha": 98.13,
 "qtd_cocaina": 0,
 "qtd_crack": 0,
 "sexo": "feminino",
 "reincidente": "não informado",
 "decisao": "parcialmente procedente",
 "tipo_pena": "aberto",
 "tempo": 0
}


In [85]:
print(saida)

{'escopo': <Escopo.SIM: 'sim'>, 'maconha': 'sim', 'cocaina': 'nao', 'crack': 'nao', 'qtd_maconha': 98.13, 'qtd_cocaina': 0.0, 'qtd_crack': 0.0, 'sexo': 'feminino', 'reincidente': 'nao informado', 'decisao': 'procedente', 'tipo_pena': 'nao informado', 'tempo': 0.0}
